### Collab

#### git clone

In [ ]:
#!git clone https://github.com/<your name>/<your project>.git
#!git clone https://github.com/leaguilar/AIknowthatfeel/ colab
#pip install -r requirements.txt
#import sys
#sys.path.append('colab')

### Importamos las librerias y el dataset

In [1]:
import numpy as np
import os
import shutil
import datetime

In [2]:
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Embedding, LSTM, Dense
from keras.callbacks import TensorBoard
from keras.preprocessing.text import one_hot
import pandas as pd
import pickle

Using TensorFlow backend.


In [3]:
from libs.workshop_libs.tensorboard import TrainValTensorBoard

### Importar el dataset

In [56]:
#Ugly Example preprocessing
pad_char=0
start_char=1
oov_char=2
index_from=3

with open('models/wordDictionary.pkl', 'rb') as handle:
    wordDict = pickle.load(handle)

def encodeList(text_list,vocab_size):
    encoded=[start_char]
    for w in text_list:
        val=wordDict.get(w,oov_char-index_from)+index_from
        if val>=vocab_size:
            val=vocab_size-1
        encoded.append(val)
    #print(encoded)
    return encoded
       
def encodeData(data,vocab_size = 5000):
    data_out=[]
    for index, row in data.iterrows():
        text=(row.iloc[0]+' ### '+row.iloc[1]+' ### '+row.iloc[2])
        text_list=text.split()
        encoded=encodeList(text_list,vocab_size)
        data_out.append(encoded)
    return np.array(data_out)
def encodedLabels(data):
    data_out=[]
    for index, row in data.iterrows():
        if row.iloc[0] == 'others':
            data_out.append([1,0,0,0])
        elif row.iloc[0] == 'happy':
            data_out.append([0,1,0,0])
        elif row.iloc[0] == 'angry':
            data_out.append([0,0,1,0])
        elif row.iloc[0] == 'sad':
            data_out.append([0,0,0,1])
        else:
            raise Exception()
    return np.array(data_out).astype(int)
def prepare_data(x_data,max_conv_length):
    x_data = sequence.pad_sequences(x_data, maxlen=max_conv_length)
    return x_data
def loadAndPrepareData(fname,vocab_size=5000,max_conv_length=50):
    data = pd.read_csv(fname,sep='	',header=None)
    x_data_raw=data.filter([1,2,3], axis=1).copy(deep=True)
    y_data_raw=data.filter([4], axis=1).copy(deep=True)  
    x_data = encodeData(x_data_raw,vocab_size)
    y_data = encodedLabels(y_data_raw)
    x_data = prepare_data(x_data,max_conv_length)                       
    return x_data,y_data

In [59]:
vocab_size=5000
max_conv_length=50

In [60]:
x_train,y_train=loadAndPrepareData('data_train/train.txt',vocab_size,max_conv_length)
x_dev,y_dev=loadAndPrepareData('data_dev/dev.txt',vocab_size,max_conv_length)
x_train

array([[   0,    0,    0, ...,   92,  182,    6],
       [   0,    0,    0, ...,  151,  688,    4],
       [   0,    0,    0, ...,  221,    4,  396],
       ...,
       [   0,    0,    0, ...,   36, 1145,    6],
       [   0,    0,    0, ..., 1584,    3,  585],
       [   0,    0,    0, ...,  204,    9,   11]])

In [61]:
len(x_train[0])

50

In [67]:
x_train[0][:40]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   1,  91,  18, 625,   5,  33, 186,   3, 781,  62,  16,
         5])

In [63]:
len(x_train)

30160

In [68]:
#QUITAR
max_data=5000
x_train=x_train[0:max_data]
y_train=y_train[0:max_data]
x_dev=x_dev[0:max_data]
y_dev=y_dev[0:max_data]

### Revisemos el dataset

In [11]:
## Cargamos tensorboard

In [69]:
%load_ext tensorboard
#%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [90]:
#!kill 4430

"kill" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [14]:
#!ps aux | grep -i tb

In [101]:
logs_base_dir = ".\\logs"
print(shutil.rmtree(logs_base_dir))

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '.\\logs'

In [102]:
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

Reusing TensorBoard on port 6006 (pid 39336), started 0:02:05 ago. (Use '!kill 39336' to kill it.)

In [17]:
### Uniformizemos los datos, truncados y padding

In [72]:
def create_model(top_words,max_review_length):
    embedding_vector_length = 32
    model = Sequential()
    model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
    model.add(LSTM(100))
    model.add(Dense(4, activation='sigmoid'))
    print(model.summary())
    return model

In [73]:
def train_model(model,x_train,y_train,x_dev,y_dev):
        logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy']
                     )
        model.fit(x_train,
                  y_train,
                  batch_size=64, #64,
                  validation_data=(x_dev, y_dev),
                  nb_epoch=10,
                  callbacks=[TrainValTensorBoard(logdir, 
                                                 histogram_freq=1,
                                                 write_graph=True
                                                )]
                 )

In [74]:
print(x_train.shape)
print(x_dev.shape)

(5000, 50)
(2755, 50)


In [85]:
model=create_model(vocab_size,max_conv_length)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 32)            160000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 404       
Total params: 213,604
Trainable params: 213,604
Non-trainable params: 0
_________________________________________________________________
None


In [103]:
train_model(model,x_train,y_train,x_dev,y_dev)

c:\users\flaquitqm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 5000 samples, validate on 2755 samples


W0724 15:04:07.702847 15432 deprecation_wrapper.py:119] From c:\users\flaquitqm\appdata\local\programs\python\python37\lib\site-packages\keras\callbacks.py:796: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W0724 15:04:07.717810 15432 deprecation_wrapper.py:119] From c:\users\flaquitqm\appdata\local\programs\python\python37\lib\site-packages\keras\callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.



Epoch 1/10
5000/5000 [==============================] - 5s 941us/step - loss: 1.2489 - acc: 0.5004 - val_loss: 0.8691 - val_acc: 0.8490
Epoch 2/10
5000/5000 [==============================] - 4s 721us/step - loss: 1.0556 - acc: 0.5230 - val_loss: 0.6426 - val_acc: 0.8599
Epoch 3/10
5000/5000 [==============================] - 4s 742us/step - loss: 0.6438 - acc: 0.7526 - val_loss: 0.4254 - val_acc: 0.8595
Epoch 4/10
5000/5000 [==============================] - 4s 750us/step - loss: 0.4173 - acc: 0.8552 - val_loss: 0.4753 - val_acc: 0.8240
Epoch 5/10
5000/5000 [==============================] - 4s 742us/step - loss: 0.3176 - acc: 0.8900 - val_loss: 0.5953 - val_acc: 0.7681
Epoch 6/10
5000/5000 [==============================] - 4s 778us/step - loss: 0.2810 - acc: 0.9060 - val_loss: 0.3914 - val_acc: 0.8617
Epoch 7/10
5000/5000 [==============================] - 4s 741us/step - loss: 0.2133 - acc: 0.9306 - val_loss: 0.4513 - val_acc: 0.8330
Epoch 8/10
5000/5000 [==========================

### Evaluamos el modelo

In [104]:
scores = model.evaluate(x_dev, y_dev, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2755/2755 [==============================] - 1s 238us/step
Accuracy: 82.61%


### Guardamos el modelo

In [105]:
outpath='models/'
os.makedirs(outpath, exist_ok=True)

In [106]:
model.save(outpath+"model.h5")

In [107]:
# serialize model to JSON
model_json = model.to_json()
with open(outpath+"model.json", "w") as json_file:
    json_file.write(model_json)

In [108]:
from keras.models import load_model

In [109]:
model2=load_model("models/model.h5")

In [110]:
scores = model2.evaluate(x_dev, y_dev, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2755/2755 [==============================] - 1s 427us/step
Accuracy: 82.61%


In [111]:
print(model.metrics_names)

['loss', 'acc']


### ADDENDUM: One hot encoder

In [47]:
from keras.preprocessing.text import one_hot

In [48]:
texto=["Esta es una frase","Otra frase mas larga", "todo esto viene de nuestro dataset de interes"]

In [49]:
vocab_size = 50
palabras_vectorizadas = [one_hot(d, vocab_size) for d in texto]
print(palabras_vectorizadas)

[[16, 3, 36, 32], [19, 32, 10, 42], [45, 31, 39, 47, 8, 46, 47, 41]]


### Problemas con numpy nuevos y keras

In [50]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
#(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [51]:
# restore np.load for future normal usage
np.load = np_load_old

In [52]:
### Otra solucion

In [53]:
#!pip uninstall numpy

In [54]:
#!pip install numpy==1.16.1